In [87]:
import pandas as pd
import numpy as np
import datetime
import string
import os

In [88]:
DATA_PATH = '../dataset/'
RAW_DATA = os.path.join(DATA_PATH, 'raw_data.csv')
OUT_DATA = os.path.join(DATA_PATH, 'out.csv')

In [89]:
dataset = pd.read_csv(RAW_DATA, keep_default_na=False)

In [90]:
dataset = dataset.rename(columns={"GIOI.x": "Female"})
s = dataset['Female'].to_numpy()

for i in range(len(s)):
    if s[i] == 'Nam' or s[i] == 'M':
        dataset.at[i,'Female'] = 0
    elif s[i] == 'Nu' or s[i] == 'F':
        dataset.at[i,'Female'] = 1

In [91]:
s = dataset['Female'].to_numpy()
male = 0
female = 0
total = dataset.shape[0]
for i in s:
    if i == 0:
        male += 1
    elif i == 1:
        female += 1
unknown = total - male - female
gender = pd.DataFrame([[total,male,female,unknown]], columns=['Total','Male','Female','Unknown'])
gender.head()

,Total,Male,Female,Unknown
0,2072,1297,775,0


In [92]:
from pathlib import Path  
filepath = Path(os.path.join(DATA_PATH, 'gender.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)
gender.to_csv(filepath, index=False)

In [93]:
dataset = dataset.drop(columns=['STT', 'ID', 'Name', 'SODIENTHOAI', 'Fisrt of Department',
                                'Department of out hospital', 'Department of test'])

In [94]:
month_name = ['Jan', 'Feb', 'Mar', 'Apr',
              'May', 'Jun', 'Jul', 'Aug',
              'Sep', 'Oct', 'Nov', 'Dec']

def to_datetime(column_name):
    s = dataset[column_name].to_numpy(dtype = 'str')
    for i in range(len(s)):
        if s[i] == 'nan':
            continue
        if '-' in s[i]:
            date = s[i].split(sep='-')
        elif '/' in s[i]:
            date = s[i].split(sep='/')
        for j in range(len(month_name)):
            if date[1] == month_name[j]:
                date[1] = j+1
        date = [int(x) for x in date]
        if date[0] > 2000: # date[0] is year
            x = datetime.date(date[0],date[1],date[2])
        else:
            if date[2] < 2000:
                date[2] += 2000
            x = datetime.date(date[2],date[1],date[0])
        dataset.at[i,column_name] = x

dataset = dataset.rename(columns={"NGAYSINH.x":"dob"})
to_datetime('dob')
dataset = dataset.rename(columns={"date of admission":"admission_date"})
to_datetime('admission_date')

In [95]:
dataset.insert(7, "age_at_admission", 0)

In [96]:
for i in range(dataset.shape[0]):
    x = dataset.at[i,'dob']
    y = dataset.at[i,'admission_date']
    if pd.isna(dataset.at[i,'dob']) == False and pd.isna(dataset.at[i,'admission_date']) == False:
        z = y - x
        dataset.at[i,"age_at_admission"] = z.days // 30

In [97]:
dataset = dataset.rename(columns={"Distric":"district"})
dis = dataset['district'].to_numpy(dtype='str')

wrong_dis = ["Huyen V_ Quang","ph_ c_","Huyen Hi_p H","Kim B_ng","Huyen H__ng Kh?","Nam Tr_c","Th__ng Xu","T__ng D__ng","Huyen C_ Kuin",
"B_m S_n","Y?n Khnh","Bnh Xuy?n","Huyen Kim __ng","Xun Tr__ng","Tam D__ng","Huyen Qu_nh L_u","Thi Xa Thai Ho?a","Thi Xa Thai Ho`a",
"TP B_c Giang","L__ng S_n","Y?n L_c","Ung B","Huyen Nng C_ng","Kim Thnh","Huyen Di_n Ch","T_ k_","Gia Vi_n","Giao Th_y","TP. Thi Nguy?n",
"L_p Th_ch","L_c Nam","Huyen T_ K_","tp L_ng S_n","tp. Ninh Bnh","Vi_t Tr","Huyen C_m Kh?","Phc Thnh","B_c Quang","Huyen Thanh Ch__ng",
"H_ H","Huyen Bnh Giang","Mai S_n","Huyen Tam Nâ¢ng","Th_y Nguy?n","Tp Thi Bnh","Ngh_a H_ng","_inh H","Thnh ph_ H_i D__ng","TP. Ninh Bnh",
"Ch Linh","TP. Nam __nh","TP.H_ng Y?n","Kinh M_n","Thanh Mi_n","H_ng Nguy?n","Nam Sch","Huyen Ngh_a _","H_u L_c","Nam c__ng","Quang S_n",
"Phc Y?n","Gia Bnh","Huyen _ng H_ng","Y?n M_","ån Thi","V_nh t__ng","Huyen V_n L","L_c H","__ S_n","H Phong","__ng T"]
correction = ["Huyen Vu Quang","Phu Cu","Huyen Hiep Hoa","Kim Bang","Huyen Huong Khe","Nam Truc","Thuong Xuan","Tuong Duong","Huyen Cu Kuin",
"Bim Son","Yen Khanh","Binh Xuyen","Huyen Kim Dong","Xuan Truong","Tam Duong","Huyen Quynh Luu","Thi Xa Thai Hoa","Thi Xa Thai Hoa",
"TP Bac Giang","Luong Son","Yen Lac","Uong Bi","Huyen Nong Cong","Kim Thanh","Huyen Dien Chau","Tu ky","Gia Vien","Giao Thuy","TP Thai Nguyen",
"Lap Thach","Luc Nam","Huyen Tu Ky","TP Lang Son","TP Ninh Binh","Viet Tri","Huyen Cam Khe","Phuc Thanh","Bac Quang","Huyen Thanh Chuong",
"Ha Hoa","Huyen Binh Giang","Mai Son","Huyen Tam Nong","Thuy Nguyen","TP Thai Binh","Nghia Hung","Dinh Hoa","TP Hai Duong","TP Ninh Binh",
"Chi Linh","TP Nam Dinh","TP Hung Yen","Kinh Mon","Thanh Mien","Hung Nguyen","Nam Sach","Huyen Nghia Dan","Hau Loc","Nam Cuong","Quang Son",
"Phuc Yen","Gia Binh","Huyen Dong Hung","Yen My","An Thi","Vinh Tuong","Huyen Van Lam","Loc Ha","Do Son","Hong Phong","Dong Tam"]

for i in range(len(dis)):
    if dis[i] == 'nan':
        dataset.at[i,'district'] = ''
        continue
    for j in range(len(wrong_dis)):
        if dis[i] == wrong_dis[j]:
            dis[i] = correction[j]
            break
    word_dis = dis[i].split(sep = ' ')
    if word_dis[0] == 'Huy_n':
        word_dis[0] = 'Huyen'
    if word_dis[0] == 'TX':
        word_dis[0] = 'Thi Xa'
    new_dis = ' '.join(word_dis)
    dataset.at[i,'district'] = new_dis

In [98]:
dis = dataset['district'].to_numpy(dtype='str')

for i in range(len(dis)):
    if dis[i] == '':
        continue
    dis[i] = string.capwords(dis[i])
    dataset.at[i,'district'] = dis[i]

In [99]:
dataset = dataset.rename(columns={"Province": "province"})
pro = dataset['province'].to_numpy(dtype='str')

for i in range(len(dis)):
    if pro[i] == 'nan':
        continue
    pro[i] = string.capwords(pro[i])
    dataset.at[i,'province'] = pro[i]

In [100]:
dataset = dataset.rename(columns={"Community": "commune"})
comm = dataset['commune'].to_numpy(dtype='str')

for i in range(len(comm)):
    if comm[i] == 'nan':
        continue
    s = ""
    for j in comm[i]:
        if (j>='a' and j<='z') or (j>='A' and j<='Z') or j ==' ' or (j>='0' and j<='9'):
            s += j
    dataset.at[i,'commune'] = s

In [101]:
comm = dataset['commune'].to_numpy(dtype='str')

for i in range(len(comm)):
    if comm[i] == '':
        continue
    comm[i] = string.capwords(comm[i])
    dataset.at[i,'commune'] = comm[i]

In [102]:
dataset.insert(5,"complete_patient_address",np.nan)

In [103]:
for i in range(dataset.shape[0]):
    commune_i = dataset.at[i,'commune']
    district_i = dataset.at[i,'district']
    province_i = dataset.at[i,'province']
    address = []
    if commune_i != '':
        address.append(commune_i)
    if district_i != '':
        address.append(district_i)
    if province_i != '':
        address.append(province_i)
    full_address = ', '.join(address)
    dataset.at[i,'complete_patient_address'] = full_address

In [104]:
dataset = dataset.rename(columns = {"date of out hospital":"discharge_date"})
to_datetime('discharge_date')

In [105]:
dataset.insert(10,'duration_of_stay',np.nan)

In [106]:
for i in range(dataset.shape[0]):
    x = dataset.at[i,'admission_date']
    y = dataset.at[i,'discharge_date']
    if pd.isna(x) == False and pd.isna(y) == False:
        z = y - x
        dataset.at[i,"duration_of_stay"] = z.days

In [107]:
dataset = dataset.drop(columns = ['LOAIXN','Note of time stay in hospital'])

In [108]:
dataset = dataset.rename(columns = {'date of test':'test_date','date of result':'testresult_date'})
to_datetime('test_date')
to_datetime('testresult_date')

In [109]:
dataset = dataset.rename(columns = {'HAI':'healthcare_associated_infection','RSV':'respiratory_syncytical_virus','ADENO':'adenovirus',
'Pertussis':'pertussis','oxygen':'oxygen_cannula','cipap':'CPAP','ventilator':'conventional_mechanical_ventilation',
'HFO ventilator':'hfo_ventilation','ECMOY:AL':'ECMO'})

def YN_to_binary(column_name):
    s = dataset[column_name].to_numpy(dtype='str')
    for i in range(len(s)):
        if s[i] == '' or s[i] == 'nan':
            dataset.at[i,column_name] = 0
            continue
        ls = s[i].lower()
        if ls == 'yes':
            dataset.at[i,column_name] = 1
        elif ls == 'no' or ls == 'na':
            dataset.at[i,column_name] = 0

YN_to_binary('oxygen_cannula')
YN_to_binary('CPAP')
YN_to_binary('pertussis')
YN_to_binary('healthcare_associated_infection')
YN_to_binary('respiratory_syncytical_virus')
YN_to_binary('adenovirus')
YN_to_binary('conventional_mechanical_ventilation')
YN_to_binary('hfo_ventilation')
YN_to_binary('ECMO')

In [110]:
dataset['ECMO'].head

<bound method NDFrame.head of 0       0
1       0
2       0
3       0
4       0
       ..
2067    0
2068    0
2069    0
2070    0
2071    0
Name: ECMO, Length: 2072, dtype: object>

In [111]:
stat = dataset['Stage of discharge'].to_numpy(dtype = 'str')
print(np.unique(stat))
res = dataset['result of discharge'].to_numpy(dtype = 'str')
print(np.unique(res))

['BN khong nhap vien' 'Bo ve' 'Chuyen vien' 'Ra vi<U+1EC7>n' 'Ra vien'
 'Tu vong' 'Xin ve' 'chua ra vien' 'do giam' 'do, giam ' 'khoi' 'tu vong']
['Chuyen  vien' 'Do/Giam' 'Khoi' 'Khong thay doi' 'Nang hon' 'Tu vong'
 'chua ra vien' 'do giam' 'do, giam ' 'khoi' 'tu vong'
 'Ð<U+1EE1>/Gi<U+1EA3>m']


In [112]:
diseases = pd.DataFrame()
def diseases_data(column_name):
    s = dataset[column_name].to_numpy(dtype = 'int')
    y = 0
    n = 0
    for i in s:
        if i == 1:
            y += 1
        elif i == 0:
            n += 1
    if y + n != len(s):
        print('nan exists')
    row = diseases.shape[1]
    diseases.insert(row,column_name,[n,y])
diseases_data('adenovirus')
diseases_data('pertussis')
diseases_data('healthcare_associated_infection')
diseases_data('respiratory_syncytical_virus')
diseases.head()
diseases.rename(index={0:'No',1:'Yes'})


,adenovirus,pertussis,healthcare_associated_infection,respiratory_syncytical_virus
No,1977,2055,2054,2042
Yes,95,17,18,30


In [113]:
from pathlib import Path  
filepath = Path(os.path.join(DATA_PATH, 'diseases.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)  
diseases.to_csv(filepath, index = True)

In [114]:
from datetime import date
def datetime_to_date(column_name):
    for i in range(dataset.shape[0]):
        x = dataset.at[i,column_name]
        y = pd.to_datetime(x)
        z = y.date()
        #print(z)
        dataset.at[i,column_name] = z

datetime_to_date('dob')
datetime_to_date('admission_date')
datetime_to_date('discharge_date')
datetime_to_date('test_date')
datetime_to_date('testresult_date')

In [115]:
dataset = dataset.drop(columns = ['dialysis','Catheter','Onsetto out hospital','ghi chu nguon'])

In [116]:
vacc = dataset['Vaccination'].to_numpy(dtype = 'str')
print(np.unique(vacc))
unique_status = ['','0','1','1 (12/2/2018)','1 (12/22/2018)','2','3','99', '?a khoa qu?c t? H?i Phòng, ?i?u tr? ph?i','d<U+1EE7> mui',
'kham bs lan anh','tiêm 15/9']
convert_status = ['NA',0,1,1,1,2,3,'NA','?','?',2,'?','?']
for i in range(len(vacc)):
    x = vacc[i]

['' '0' '1' '1 (12/2/2018)' '1 (12/22/2018)' '2' '3' '99'
 '?a khoa qu?c t? H?i Phòng, ?i?u tr? ph?i' 'NA' 'd<U+1EE7> mui'
 'kham bs lan anh' 'tiêm 15/9']


In [117]:
dataset = dataset.rename(columns={'Code source': 'place_of_exposure'})

dataset['place_of_exposure'] = dataset['place_of_exposure'].str.lower()

for i in range(len(dataset['place_of_exposure'])):
    code = dataset.at[i, 'place_of_exposure']
    if code in ['suppected nch', 'definitely nch']:
        dataset.at[i, 'place_of_exposure'] = 'NHP'
    elif code in ['suppected other h', 'suppected  other h', 'definitely other h']:
        dataset.at[i, 'place_of_exposure'] = 'Another hospital'
    elif code in ['community']:
        continue

In [118]:
# Calculate number of patients in each place of exposure
in_nhp, in_another_hospital, in_community = 0, 0, 0
total = dataset.shape[0]

for i in dataset['place_of_exposure']:
    if i == 'NHP':
        in_nhp += 1
    elif i == 'Another hospital':
        in_another_hospital += 1
    elif i == 'community':
        in_community += 1
unknown = total - in_nhp - in_another_hospital - in_community
poe = pd.DataFrame([[total, in_nhp, in_another_hospital, in_community, unknown]], columns=['Total', 'NHP', 'Another hospital', 'Community', 'Unknown'])

In [119]:
from pathlib import Path  
filepath = Path(os.path.join(DATA_PATH, 'place_of_exposure.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)  
poe.to_csv(filepath, index=True)

In [120]:
from pathlib import Path  
filepath = Path(OUT_DATA)
filepath.parent.mkdir(parents=True, exist_ok=True)
dataset.to_csv(filepath, index=False)